# Fraud Detection - Model Training and Evaluation

This notebook focuses on training and evaluating machine learning models for the IEEE-CIS Fraud Detection task.

## Why Tree-Based Models?

Tree-based ensemble methods are particularly well-suited for fraud detection due to:

1. **Robustness to Feature Distributions**: No assumptions about normality; handles skewed transaction amounts naturally
2. **Missing Value Handling**: Native support for missing values (especially LightGBM, XGBoost)
3. **Feature Interactions**: Automatically captures non-linear relationships and complex interactions
4. **Imbalanced Data**: Built-in mechanisms (`scale_pos_weight`, `is_unbalance`) for class imbalance
5. **Correlation Insensitivity**: Unlike linear models, not affected by multicollinearity
6. **Outlier Robustness**: Split-based decisions are less sensitive to extreme values
7. **Mixed Feature Types**: Handles both numerical and encoded categorical features seamlessly

**Models Compared**:
1. Random Forest - Bagging ensemble, robust baseline
2. XGBoost - Gradient boosting with regularization
3. LightGBM - Fast gradient boosting, leaf-wise growth
4. CatBoost - Native categorical handling, ordered boosting
5. Stacking Ensemble - Combines base models via weighted averaging

**Evaluation Strategy**: 
- K-Fold Cross-Validation on training set for model stability assessment
- Single evaluation on temporal hold-out test set for unbiased performance
- Primary metric: ROC-AUC (standard for fraud detection)

In [1]:
# =============================================================================
# Import Libraries
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import time
warnings.filterwarnings('ignore')

# Sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    roc_auc_score, roc_curve, auc, average_precision_score,
    precision_score, recall_score, f1_score
)
from sklearn.preprocessing import StandardScaler

# Boosting models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Custom functions
import sys
ROOT = Path.cwd().parent
sys.path.append(str(ROOT / "functions"))

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Data & Configuration

In [2]:
# =============================================================================
# Load Preprocessed Data from EDA.ipynb
# =============================================================================
import pickle

DATA = ROOT / "data"

# Load DataFrames
train_df = pd.read_parquet(DATA / "train_preprocessed.parquet")
test_df = pd.read_parquet(DATA / "test_preprocessed.parquet")

# Load feature lists
with open(DATA / "feature_lists.pkl", 'rb') as f:
    feature_lists = pickle.load(f)

filtered_features = feature_lists['filtered_features']
categorical_features = feature_lists['categorical_for_model']
strong_features = feature_lists['strong_features']
moderate_features = feature_lists['moderate_features']

print(f"✅ Data loaded successfully!")
print(f"   Train shape: {train_df.shape}")
print(f"   Test shape: {test_df.shape}")
print(f"   Features for modeling: {len(filtered_features)}")

✅ Data loaded successfully!
   Train shape: (472432, 195)
   Test shape: (118108, 195)
   Features for modeling: 175


In [3]:
# =============================================================================
# Prepare Features and Target
# =============================================================================
X_train = train_df[filtered_features].copy()
y_train = train_df['isFraud'].copy()
X_test = test_df[filtered_features].copy()
y_test = test_df['isFraud'].copy()

# =============================================================================
# Handle Infinity and Missing Values
# =============================================================================
inf_cols = X_train.columns[X_train.isin([np.inf, -np.inf]).any()].tolist()
if inf_cols:
    print(f" Found infinity in {len(inf_cols)} columns: {inf_cols}")

X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

MISSING_FLAG = -999  # yarışma birincisinin uygulamasında -999 kullanılmış böylece ağaç algoritmalarında bu kayıtlar direkt ayrılıyor
X_train = X_train.fillna(MISSING_FLAG)
X_test = X_test.fillna(MISSING_FLAG)

print(f"   Data prepared: X_train {X_train.shape}, X_test {X_test.shape}")
print(f"   Train fraud rate: {y_train.mean()*100:.2f}%")
print(f"   Test fraud rate: {y_test.mean()*100:.2f}%")

   Data prepared: X_train (472432, 175), X_test (118108, 175)
   Train fraud rate: 3.51%
   Test fraud rate: 3.44%


### Temporal Split Justification

**Why Temporal vs Random Split:**

In fraud detection, using a **temporal (time-based) split** is critical because:

1. **Concept Drift**: Fraudsters continuously adapt their strategies over time, making historical patterns less predictive of future fraud.
2. **Data Leakage Prevention**: Random splits would allow "future" information to leak into training, artificially inflating performance metrics.
3. **Realistic Evaluation**: Models trained on past data must predict future fraud patterns, mirroring real-world deployment.

**Academic Justification:**

- *Krawczyk et al. (2017), "Ensemble learning for data stream analysis: A survey", Information Fusion*: Highlights the importance of temporal ordering in streaming data classification tasks.
- *Dal Pozzolo et al. (2014), "Learned lessons in credit card fraud detection from a practitioner perspective", Expert Systems with Applications*: Demonstrates that temporal splits provide more realistic performance estimates in fraud detection.

**Implication**: CV scores (computed on training data) may differ from test set performance because the test set represents a **future time period** with potentially different fraud patterns. This is expected behavior and reflects real-world deployment conditions.

In [4]:
# # =============================================================================
# # Visualize Train/Test Split (Temporal)
# # =============================================================================
# fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# # 1. Dataset Size Comparison
# ax1 = axes[0]
# sizes = [len(X_train), len(X_test)]
# labels = ['Train (80%)', 'Test (20%)']
# colors = ['#3498db', '#e74c3c']
# bars = ax1.bar(labels, sizes, color=colors, edgecolor='black', linewidth=1.2)
# ax1.set_ylabel('Number of Samples', fontsize=11)
# ax1.set_title('Dataset Split', fontsize=12, fontweight='bold')
# for bar, size in zip(bars, sizes):
#     ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, 
#              f'{size:,}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# # 2. Fraud Rate Comparison
# ax2 = axes[1]
# fraud_rates = [y_train.mean()*100, y_test.mean()*100]
# bars2 = ax2.bar(labels, fraud_rates, color=colors, edgecolor='black', linewidth=1.2)
# ax2.set_ylabel('Fraud Rate (%)', fontsize=11)
# ax2.set_title('Fraud Rate by Split', fontsize=12, fontweight='bold')
# ax2.axhline(y=y_train.mean()*100, color='gray', linestyle='--', alpha=0.5, label='Train Rate')
# for bar, rate in zip(bars2, fraud_rates):
#     ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
#              f'{rate:.2f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

# # 3. Class Distribution (Stacked)
# ax3 = axes[2]
# train_counts = [len(y_train) - y_train.sum(), y_train.sum()]
# test_counts = [len(y_test) - y_test.sum(), y_test.sum()]
# x_pos = np.arange(2)
# width = 0.35

# bars_normal = ax3.bar(x_pos - width/2, [train_counts[0], test_counts[0]], width, 
#                        label='Normal', color='#2ecc71', edgecolor='black')
# bars_fraud = ax3.bar(x_pos + width/2, [train_counts[1], test_counts[1]], width, 
#                       label='Fraud', color='#e74c3c', edgecolor='black')
# ax3.set_xticks(x_pos)
# ax3.set_xticklabels(['Train', 'Test'])
# ax3.set_ylabel('Count', fontsize=11)
# ax3.set_title('Class Distribution', fontsize=12, fontweight='bold')
# ax3.legend(loc='upper right')
# ax3.set_yscale('log')  # Log scale due to imbalance

# plt.tight_layout()
# plt.show()

# # Summary table
# print("\n" + "="*60)
# print("DATA SPLIT SUMMARY (Temporal Split: Train=Past, Test=Future)")
# print("="*60)
# print(f"{'Dataset':<12} {'Samples':>12} {'Normal':>12} {'Fraud':>10} {'Fraud %':>10}")
# print("-"*60)
# print(f"{'Train':<12} {len(X_train):>12,} {int(len(y_train)-y_train.sum()):>12,} {int(y_train.sum()):>10,} {y_train.mean()*100:>9.2f}%")
# print(f"{'Test':<12} {len(X_test):>12,} {int(len(y_test)-y_test.sum()):>12,} {int(y_test.sum()):>10,} {y_test.mean()*100:>9.2f}%")
# print("="*60)

## 2. Model Definitions & Stacking Configuration

In [5]:
# =============================================================================
# CONFIGURATION - Change these variables to control the pipeline
# =============================================================================

# SMOTE Configuration:
#   None          - No resampling (use built-in class weights)
#   'smote'       - Standard SMOTE oversampling
#   'smote_tomek' - SMOTE + Tomek links (cleaner decision boundaries)

SMOTE_METHOD = None  # <-- CHANGE THIS TO SWITCH SMOTE METHOD

# Stacking Configuration:
#   'logistic'  - Logistic Regression as meta-model (recommended for probability calibration)
#   'ridge'     - Ridge Classifier as meta-model (more regularization)
#   'weighted'  - No meta-model, weighted average based on CV performance

STACKING_METHOD = 'weighted'  # <-- CHANGE THIS TO SWITCH STACKING METHOD

print(f"📋 Configuration:")
print(f"   SMOTE Method: {SMOTE_METHOD if SMOTE_METHOD else 'None (using class weights)'}")
print(f"   Stacking Method: {STACKING_METHOD}")

📋 Configuration:
   SMOTE Method: None (using class weights)
   Stacking Method: weighted


In [6]:
# =============================================================================
# Model Definitions (automatically adapts to SMOTE setting)
# =============================================================================

# Calculate class imbalance ratio (only used when SMOTE is disabled)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

if SMOTE_METHOD is None:
    # Use built-in class weights when SMOTE is disabled
    print(f"⚖️ Using class weights (imbalance ratio: {scale_pos_weight:.2f})")
    
    BASE_MODELS = {
        'RandomForest': RandomForestClassifier(
            n_estimators=200,
            max_depth=15,
            min_samples_split=10,
            min_samples_leaf=5,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        ),
        'XGBoost': XGBClassifier(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=scale_pos_weight,
            eval_metric='auc',
            random_state=42,
            n_jobs=-1
        ),
        'LightGBM': LGBMClassifier(
            n_estimators=300,
            max_depth=8,
            learning_rate=0.05,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            is_unbalance=True,
            random_state=42,
            n_jobs=-1,
            verbose=-1
        ),
        'CatBoost': CatBoostClassifier(
            iterations=300,
            depth=6,
            learning_rate=0.05,
            auto_class_weights='Balanced',
            eval_metric='AUC',
            random_seed=42,
            verbose=0
        )
    }
else:
    # No class weights needed when SMOTE balances the data
    print(f"⚖️ SMOTE active - class weights disabled (data is balanced)")
    
    BASE_MODELS = {
        'RandomForest': RandomForestClassifier(
            n_estimators=200,
            max_depth=15,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42,
            n_jobs=-1
        ),
        'XGBoost': XGBClassifier(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.05,
            subsample=0.8,
            colsample_bytree=0.8,
            eval_metric='auc',
            random_state=42,
            n_jobs=-1
        ),
        'LightGBM': LGBMClassifier(
            n_estimators=300,
            max_depth=8,
            learning_rate=0.05,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbose=-1
        ),
        'CatBoost': CatBoostClassifier(
            iterations=300,
            depth=6,
            learning_rate=0.05,
            eval_metric='AUC',
            random_seed=42,
            verbose=0
        )
    }

print(f"✅ {len(BASE_MODELS)} base models defined")

⚖️ Using class weights (imbalance ratio: 27.46)
✅ 4 base models defined


### Apply SMOTE Resampling (Configured Above)

The `SMOTE_METHOD` variable controls how class imbalance is handled:

| Setting | Description | When to Use |
|---------|-------------|-------------|
| `None` | No resampling, use built-in class weights | Default, usually works well |
| `'smote'` | Synthetic Minority Over-sampling | When class weights underperform |
| `'smote_tomek'` | SMOTE + Tomek links cleanup | For cleaner decision boundaries |

**Note:** When SMOTE is active, model class weights are automatically disabled since the data is already balanced.

In [7]:
# =============================================================================
# Apply SMOTE (if configured)
# =============================================================================

# Store original data for reference
X_train_original = X_train.copy()
y_train_original = y_train.copy()

if SMOTE_METHOD is not None:
    from imblearn.over_sampling import SMOTE
    from imblearn.combine import SMOTETomek
    
    print(f"Applying {SMOTE_METHOD.upper()} to training data...")
    print(f"Before: {len(X_train):,} samples (Fraud: {y_train.sum():,}, Rate: {y_train.mean()*100:.2f}%)")
    
    if SMOTE_METHOD == 'smote':
        sampler = SMOTE(
            sampling_strategy=0.5,  # Ratio of minority to majority
            random_state=42,
            k_neighbors=5,
            n_jobs=-1
        )
    elif SMOTE_METHOD == 'smote_tomek':
        sampler = SMOTETomek(
            sampling_strategy=0.5,
            random_state=42,
            n_jobs=-1
        )
    else:
        raise ValueError(f"Unknown SMOTE method: {SMOTE_METHOD}. Use None, 'smote', or 'smote_tomek'")
    
    X_train, y_train = sampler.fit_resample(X_train, y_train)
    
    print(f"After:  {len(X_train):,} samples (Fraud: {y_train.sum():,}, Rate: {y_train.mean()*100:.2f}%)")
    
    # Visualize the resampling effect
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Before SMOTE
    ax1 = axes[0]
    counts_before = [len(y_train_original) - y_train_original.sum(), y_train_original.sum()]
    ax1.bar(['Normal', 'Fraud'], counts_before, color=['#2ecc71', '#e74c3c'], edgecolor='black')
    ax1.set_title('Before SMOTE', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Count')
    for i, v in enumerate(counts_before):
        ax1.text(i, v + 500, f'{int(v):,}', ha='center', fontweight='bold')
    
    # After SMOTE
    ax2 = axes[1]
    counts_after = [len(y_train) - y_train.sum(), y_train.sum()]
    ax2.bar(['Normal', 'Fraud'], counts_after, color=['#2ecc71', '#e74c3c'], edgecolor='black')
    ax2.set_title(f'After {SMOTE_METHOD.upper()}', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Count')
    for i, v in enumerate(counts_after):
        ax2.text(i, v + 500, f'{int(v):,}', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚙️ SMOTE disabled - using built-in class weights for imbalance handling")

⚙️ SMOTE disabled - using built-in class weights for imbalance handling


## 3. Model Training with Simplified Evaluation

## Model Evaluation Methodology

**Approach**: Hold-out temporal test set with K-Fold CV for model stability assessment.

**Rationale**:
- **CV on train (K=3)**: Estimates model stability and variance across different data subsets
- **Single test evaluation**: Unbiased performance estimate on unseen future data
- **Temporal split**: Prevents data leakage (fraudsters adapt over time)

**Academic Justification**:
- *Hastie et al. (2009), "The Elements of Statistical Learning", Chapter 7.10*: "For model assessment, an independent test set is ideal. For model selection, use cross-validation."
- This prevents "peeking" at test data during hyperparameter tuning while still providing robust stability estimates.

**Metrics**:
- **Primary**: ROC-AUC (handles class imbalance, threshold-independent)
- **Secondary**: Average Precision (PR-AUC), optimal F1-threshold metrics

In [8]:
# =============================================================================
# Simplified Model Training: CV for Stability + Single Test Evaluation
# =============================================================================
N_FOLDS = 3
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# Storage for results and trained models
results = []
trained_models = {}
test_predictions = {}

# Prepare SMOTE sampler if needed
if SMOTE_METHOD is not None:
    from imblearn.over_sampling import SMOTE
    from imblearn.combine import SMOTETomek
    
    if SMOTE_METHOD == 'smote':
        sampler = SMOTE(sampling_strategy=0.5, random_state=42, k_neighbors=5, n_jobs=-1)
    else:
        sampler = SMOTETomek(sampling_strategy=0.5, random_state=42, n_jobs=-1)
    
    X_train_fit, y_train_fit = sampler.fit_resample(X_train, y_train)
    print(f"SMOTE applied: {len(X_train_fit):,} samples (from {len(X_train):,})")
else:
    X_train_fit, y_train_fit = X_train, y_train

print(f"Training {len(BASE_MODELS)} models...")
print(f"Method: {N_FOLDS}-Fold CV (stability) + Single test evaluation")
print("=" * 70)

for model_name, model in BASE_MODELS.items():
    start_time = time.time()
    
    # Step 1: CV for stability assessment (on original train data)
    cv_scores = cross_val_score(
        model, X_train_fit, y_train_fit,
        cv=skf, scoring='roc_auc', n_jobs=-1
    )
    
    # Step 2: Train ONCE on full training set
    model.fit(X_train_fit, y_train_fit)
    trained_models[model_name] = model
    
    # Step 3: Single test prediction
    test_probs = model.predict_proba(X_test)[:, 1]
    test_predictions[model_name] = test_probs
    
    # Calculate test metrics
    test_auc = roc_auc_score(y_test, test_probs)
    test_ap = average_precision_score(y_test, test_probs)
    
    # Optimal threshold using Youden's J statistic
    fpr, tpr, thresholds = roc_curve(y_test, test_probs)
    j_scores = tpr - fpr
    optimal_idx = np.argmax(j_scores)
    optimal_thresh = thresholds[optimal_idx]
    
    # Metrics at optimal threshold
    test_pred_binary = (test_probs >= optimal_thresh).astype(int)
    precision_opt = precision_score(y_test, test_pred_binary)
    recall_opt = recall_score(y_test, test_pred_binary)
    f1_opt = f1_score(y_test, test_pred_binary)
    
    elapsed_time = time.time() - start_time
    
    # Store results
    results.append({
        'Model': model_name,
        'CV_AUC_mean': cv_scores.mean(),
        'CV_AUC_std': cv_scores.std(),
        'Test_AUC': test_auc,
        'Test_AP': test_ap,
        'Optimal_Threshold': optimal_thresh,
        'Precision_opt': precision_opt,
        'Recall_opt': recall_opt,
        'F1_opt': f1_opt
    })
    
    print(f"{model_name}: CV AUC {cv_scores.mean():.4f} +/- {cv_scores.std():.4f} | "
          f"Test AUC {test_auc:.4f} | Time: {elapsed_time:.1f}s")

# Store CV scores for stacking weights
cv_scores_dict = {r['Model']: r['CV_AUC_mean'] for r in results}

print("=" * 70)
print("Base model training complete.")

Training 4 models...
Method: 3-Fold CV (stability) + Single test evaluation
RandomForest: CV AUC 0.9345 +/- 0.0006 | Test AUC 0.8845 | Time: 248.1s
XGBoost: CV AUC 0.9404 +/- 0.0003 | Test AUC 0.8978 | Time: 1026.9s
LightGBM: CV AUC 0.9381 +/- 0.0008 | Test AUC 0.9003 | Time: 45.7s
CatBoost: CV AUC 0.9182 +/- 0.0008 | Test AUC 0.8933 | Time: 105.3s
Base model training complete.


## 4. Stacking Ensemble

Combine base model predictions using the configured stacking method.

**Simplified Approach**: Since we now train each model once on the full training set, stacking uses test predictions directly with weights based on CV performance.

In [9]:
# =============================================================================
# Simplified Stacking Ensemble
# =============================================================================

print(f"Stacking method: {STACKING_METHOD}")

if STACKING_METHOD == 'weighted':
    # Weighted average based on CV performance
    total_auc = sum(cv_scores_dict.values())
    weights = {name: auc_val / total_auc for name, auc_val in cv_scores_dict.items()}
    
    print(f"Weights: {dict((k, round(v, 4)) for k, v in weights.items())}")
    
    stacking_probs = np.zeros(len(y_test))
    for name in BASE_MODELS:
        stacking_probs += test_predictions[name] * weights[name]

elif STACKING_METHOD in ['logistic', 'ridge']:
    # Meta-model requires train predictions - use CV predictions
    print("Note: For meta-model stacking, using weighted average as fallback")
    print("(Full OOF predictions not available in simplified pipeline)")
    
    total_auc = sum(cv_scores_dict.values())
    weights = {name: auc_val / total_auc for name, auc_val in cv_scores_dict.items()}
    
    stacking_probs = np.zeros(len(y_test))
    for name in BASE_MODELS:
        stacking_probs += test_predictions[name] * weights[name]

else:
    raise ValueError(f"Unknown stacking method: {STACKING_METHOD}")

# Store stacking predictions
test_predictions['Stacking'] = stacking_probs

# Calculate stacking metrics
stacking_auc = roc_auc_score(y_test, stacking_probs)
stacking_ap = average_precision_score(y_test, stacking_probs)

# Optimal threshold for stacking
fpr, tpr, thresholds = roc_curve(y_test, stacking_probs)
j_scores = tpr - fpr
optimal_idx = np.argmax(j_scores)
optimal_thresh = thresholds[optimal_idx]

stacking_pred_binary = (stacking_probs >= optimal_thresh).astype(int)
precision_opt = precision_score(y_test, stacking_pred_binary)
recall_opt = recall_score(y_test, stacking_pred_binary)
f1_opt = f1_score(y_test, stacking_pred_binary)

# Add stacking to results
results.append({
    'Model': 'Stacking',
    'CV_AUC_mean': np.nan,
    'CV_AUC_std': np.nan,
    'Test_AUC': stacking_auc,
    'Test_AP': stacking_ap,
    'Optimal_Threshold': optimal_thresh,
    'Precision_opt': precision_opt,
    'Recall_opt': recall_opt,
    'F1_opt': f1_opt
})

print(f"\nStacking ensemble: Test AUC {stacking_auc:.4f} | AP {stacking_ap:.4f}")

Stacking method: weighted
Weights: {'RandomForest': 0.2505, 'XGBoost': 0.252, 'LightGBM': 0.2514, 'CatBoost': 0.2461}

Stacking ensemble: Test AUC 0.9027 | AP 0.4994


## 5. Model Evaluation and Comparison

## Threshold Optimization

**Method**: Youden's J statistic maximization

The optimal classification threshold is determined by maximizing Youden's J statistic:

$$J = \text{Sensitivity} + \text{Specificity} - 1 = \text{TPR} - \text{FPR}$$

This method balances false positives and false negatives, providing a principled threshold that does not rely on arbitrary values (e.g., 0.5).

**Academic Citation**: *Youden, W.J. (1950), "Index for rating diagnostic tests", Cancer, 3(1), 32-35.*

**Alternative**: F1 maximization is used when precision and recall are equally important, particularly in high-imbalance scenarios.

In [10]:
# =============================================================================
# Final Results Table (Excel-Ready)
# =============================================================================

# Create results DataFrame
results_df = pd.DataFrame(results)

# Sort by Test_AUC descending
results_df = results_df.sort_values('Test_AUC', ascending=False).reset_index(drop=True)

# Format numeric columns to 4 decimal places
numeric_cols = ['CV_AUC_mean', 'CV_AUC_std', 'Test_AUC', 'Test_AP', 
                'Optimal_Threshold', 'Precision_opt', 'Recall_opt', 'F1_opt']

for col in numeric_cols:
    results_df[col] = results_df[col].round(4)

# Display results
print("=" * 100)
print("MODEL COMPARISON RESULTS")
print("=" * 100)
print(results_df.to_string(index=False))
print("\nResults are Excel-ready (copy from output above)")

# Identify best model
best_model = results_df.iloc[0]['Model']
best_auc = results_df.iloc[0]['Test_AUC']
print(f"\nBest Model: {best_model} (Test ROC-AUC: {best_auc:.4f})")

MODEL COMPARISON RESULTS
       Model  CV_AUC_mean  CV_AUC_std  Test_AUC  Test_AP  Optimal_Threshold  Precision_opt  Recall_opt  F1_opt
    Stacking          NaN         NaN    0.9027   0.4994             0.3824         0.1631      0.7889  0.2703
    LightGBM       0.9381      0.0008    0.9003   0.5042             0.4406         0.1703      0.7704  0.2789
     XGBoost       0.9404      0.0003    0.8978   0.4889             0.3293         0.1613      0.7894  0.2679
    CatBoost       0.9182      0.0008    0.8933   0.4800             0.4826         0.1500      0.7793  0.2515
RandomForest       0.9345      0.0006    0.8845   0.4414             0.2560         0.1268      0.8246  0.2198

Results are Excel-ready (copy from output above)

Best Model: Stacking (Test ROC-AUC: 0.9027)


## 6. Optional: Feature Importance Analysis

Uncomment and run the cell below to visualize feature importance across all base models.

In [11]:
# =============================================================================
# [OPTIONAL] Feature Importance Comparison (All Base Models)
# =============================================================================
# Uncomment to visualize feature importance

# fig, axes = plt.subplots(2, 2, figsize=(16, 14))
# axes = axes.flatten()
# 
# for idx, (model_name, model) in enumerate(trained_models.items()):
#     ax = axes[idx]
#     
#     if hasattr(model, 'feature_importances_'):
#         importances = model.feature_importances_
#     elif hasattr(model, 'get_feature_importance'):
#         importances = model.get_feature_importance()
#     
#     importance_df = pd.DataFrame({
#         'Feature': filtered_features,
#         'Importance': importances
#     }).sort_values('Importance', ascending=True).tail(15)
#     
#     colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(importance_df)))
#     ax.barh(importance_df['Feature'], importance_df['Importance'], color=colors)
#     ax.set_xlabel('Importance')
#     ax.set_title(f'{model_name} - Top 15 Features', fontsize=12, fontweight='bold')
#     ax.grid(axis='x', alpha=0.3)
# 
# plt.suptitle('Feature Importance Comparison', fontsize=14, fontweight='bold', y=1.02)
# plt.tight_layout()
# plt.show()

## 7. Save Best Model and Results

In [12]:
# =============================================================================
# Save Models and Results (Optional)
# =============================================================================
import joblib

# Uncomment to save:

# Save all trained models
# models_dir = ROOT / "models"
# models_dir.mkdir(exist_ok=True)
# 
# for name, model in trained_models.items():
#     joblib.dump(model, models_dir / f"{name.lower()}_model.pkl")
# 
# print(f"Models saved to {models_dir}")

# Save results to CSV (Excel-ready)
# results_df.to_csv(ROOT / "results" / "model_comparison.csv", index=False)
# print("Results saved to model_comparison.csv")